In [16]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostClassifier
%matplotlib inline




le = preprocessing.LabelEncoder()

### un modelo precario, solo usamos el largo de mi texto.

### Cargo mi texto

In [17]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


### arreglo los que estan mal 

In [19]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.loc[train['id'].isin(ids_with_target_error),'target'] = 0

### agrego una columna con el largo del tweet

In [20]:
train['largo'] = train['text'].str.len()

In [22]:
train.sample(3)

,id,keyword,location,text,target,largo
2057,2950,danger,Jersey - C.I,Honestly tho Modibo Maiga is stealing a living...,1,137
1844,2651,crashed,Buenos Aires,MH370: Intact part lifts odds plane glided not...,1,86
5756,8216,riot,NaN,Stuart Broad Takes Eight Before Joe Root Runs ...,0,136


In [23]:
train = train [ ['id','largo', 'target' ] ]

In [26]:
train.sample(3)

,id,largo,target
1171,1688,123,1
571,826,138,0
5380,7678,131,0


In [45]:
largos = train['largo'].values.reshape(-1,1)

In [53]:
targets = train['target']

In [46]:
largos

array([[ 69],
       [ 38],
       [133],
       ...,
       [ 65],
       [137],
       [ 94]])

In [47]:
modelo = KNeighborsClassifier(n_neighbors=7)

In [48]:
modelo.fit(largos,list(train['target']))


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [50]:
preds = modelo.predict(largos)

### pruebo como va, hago mis pred sobre el test.

In [54]:
print('Accuracy Score : ' + str(accuracy_score(targets,preds)))
print('Precision Score : ' + str(precision_score(targets,preds)))
print('Recall Score : ' + str(recall_score(targets,preds)))
print('F1 Score : ' + str(f1_score(targets,preds)))

Accuracy Score : 0.577958754761592
Precision Score : 0.5085020242914979
Recall Score : 0.3858678955453149
F1 Score : 0.43877729257641923
